In [ ]:
import os 
import json 
import pandas as pd
import traceback

In [ ]:
!pip install langchain_groq

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
KEY =os.getenv("GROQ_API_KEY")

In [ ]:
# from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key = KEY,
    model="mixtral-8x7b-32768",
    temperature=0.5
)

In [ ]:
# Custom callback handler for Groq
from contextlib import contextmanager

import tiktoken



from langchain.callbacks.base import BaseCallbackHandler#+
class GroqCallbackHandler(BaseCallbackHandler):

    def __init__(self):
        super().__init__()
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.total_cost = 0.0
        self.encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")  # Using this as a proxy for Groq models

    def on_llm_start(self, serialized, prompts, **kwargs):
        print(f"Starting LLM call...")
        prompt_tokens = sum(len(self.encoding.encode(p)) for p in prompts)
        self.prompt_tokens += prompt_tokens

    def on_llm_end(self, response, **kwargs):
        if response.generations:
            completion_tokens = sum(len(self.encoding.encode(gen.text)) for gen in response.generations[0])
            self.completion_tokens += completion_tokens
            self.total_tokens = self.prompt_tokens + self.completion_tokens
            # Note: Adjust the cost calculation based on Groq's actual pricing
            self.total_cost += (self.total_tokens / 1000) * 0.0001  # Assuming $0.0001 per 1k tokens
        print(f"LLM call completed. Total tokens used: {self.total_tokens}")

    def on_chain_start(self, serialized, inputs, **kwargs):
        print(f"Starting chain: {serialized.get('name', 'Unknown')}")

    def on_chain_end(self, outputs, **kwargs):
        print(f"Chain completed. Output keys: {list(outputs.keys())}")

@contextmanager
def get_groq_callback():
    cb = GroqCallbackHandler()
    yield cb

In [ ]:
# from contextlib import contextmanager
%pip install tiktoken



In [ ]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.messages import HumanMessage
import json
import os




In [ ]:
RESPONSE_JSON = {
    "1":{
    " mcq":"Multiple Choice question",
    "options":{
        "a": "choice here",
        "b": "choice here",
        "c": "choice here",
        "d": "choice here",
        
    },
    "correct":"correct answer",
},
"2":{
    "mcq":"Multiple Choice question",
    "options":{
        "a": "choice here",
        "b": "choice here",
        "c": "choice here",
        "d": "choice here",
        
    },
    "correct":"correct answer",
},
"3":{
    "mcq":"Multiple Choice question",
    "options":{
        "a": "choice here",
        "b": "choice here",
        "c": "choice here",
        "d": "choice here",
        
    },
    "correct":"correct answer",
},  
}

In [ ]:
TEMPLATE = """
Text:{text}
You are an exper MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students {tone} tone.
Make sure the questions are not repeated and check all the questionsto be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below as use it as a guide. \
    Ensure to make {number} MCQs
    ### RESPONSE_JSON  
    
    {response_json}

"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["Text","number","Subject","Tone","response_json"],
    template=TEMPLATE
    
)

In [ ]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [ ]:
TEMPLATE2 = """
You are expert english grammarian and writer . Given a multiple choice quiz for {subject} students. \
    You need to evaluate the complexity of the question and give a complete analysis of the quiz . Only use at max 50 words for complexity.
    if the quiz is not at per with the cognitive and analytical abilities of the students,\
    update the quiz questions which needs to be changed and change the tone as it perfectly fits the students ability.
    Quiz_MCQs:
    {quiz}
    
    Check from an expert english writer of the above quiz:
    """

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [ ]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                                        output_variables=["quiz","review"],verbose=True)

In [ ]:
file_path=r"C:\Users\DELL\mcqgen\data.txt"

In [ ]:
with open(file_path,'r') as file:
    Text=file.read()

In [ ]:
print(Text)

In [ ]:
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [ ]:
# Use the chain with the custom callback
with get_groq_callback() as cb:
    response = generate_evaluate_chain({
        "text": Text,
        "number": NUMBER,  # Make sure NUMBER is defined
        "subject": SUBJECT,  # Make sure SUBJECT is defined
        "tone": TONE,  # Make sure TONE is defined
        "response_json": json.dumps(RESPONSE_JSON)
    })

In [ ]:
# Print the results and token usage
print(f"Quiz: {response['quiz']}")
print(f"Review: {response['review']}")
print(f"\nToken Usage:")
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost:.4f}")

In [ ]:
quiz=response.get("quiz")

In [ ]:
quiz = json.loads(quiz)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}:{option_value}"
            for option,option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq,"Choices": options,"Correct":correct})

In [ ]:
quiz_table_data

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv",index=False)